Link: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network_raw.py

In [1]:
import tensorflow as tf

import os
import numpy as np
import time

from alexnet import AlexNet
from datetime import datetime
from tensorflow.contrib.data import Iterator

from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/xutiancheng/anaconda2/envs/ipykernel_py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Training Parameters
learning_rate = 0.001
num_epochs = 10
batch_size = 32
display_step = 10

# Network params
dropout_rate = 0.5
num_classes = 2

# How often we want to write the tf.summary data to disk
display_step = 20


In [3]:
filewriter_path = '/tmp/finetune_alexnet/tensorboard'
checkpoint_path = '/tmp/finetune_alexnet/checkpoints'

train_file = '../0506samples.txt'
val_file = '../0506samplesval.txt'

assert(os.path.isfile(train_file))
assert(os.path.isfile(val_file))

In [4]:
if not os.path.isdir(checkpoint_path):
    os.mkdir(checkpoint_path)

In [5]:
with tf.device('/cpu:0'):
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
    val_data = ImageDataGenerator(val_file,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

    # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                       tr_data.data.output_shapes)
    next_batch = iterator.get_next()

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.


In [6]:
# Ops for initializing the two different iterators
training_init_op = iterator.make_initializer(tr_data.data)
validation_init_op = iterator.make_initializer(val_data.data)

# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, [batch_size, 2])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
model = AlexNet(x, keep_prob, 2, skip_layer=None)

# Link variable to model output
score = model.fc8

In [7]:
# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables()]

In [8]:
var_list

[<tf.Variable 'conv1/weights:0' shape=(11, 11, 3, 96) dtype=float32_ref>,
 <tf.Variable 'conv1/biases:0' shape=(96,) dtype=float32_ref>,
 <tf.Variable 'conv2/weights:0' shape=(5, 5, 48, 256) dtype=float32_ref>,
 <tf.Variable 'conv2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3/weights:0' shape=(3, 3, 256, 384) dtype=float32_ref>,
 <tf.Variable 'conv3/biases:0' shape=(384,) dtype=float32_ref>,
 <tf.Variable 'conv4/weights:0' shape=(3, 3, 192, 384) dtype=float32_ref>,
 <tf.Variable 'conv4/biases:0' shape=(384,) dtype=float32_ref>,
 <tf.Variable 'conv5/weights:0' shape=(3, 3, 192, 256) dtype=float32_ref>,
 <tf.Variable 'conv5/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'fc6/weights:0' shape=(9216, 4096) dtype=float32_ref>,
 <tf.Variable 'fc6/biases:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'fc7/weights:0' shape=(4096, 4096) dtype=float32_ref>,
 <tf.Variable 'fc7/biases:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'fc8/weights:0' shape=(4096, 2

In [9]:
# Op for calculating the loss
with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,
                                                                  labels=y))

In [10]:
# Train op
with tf.name_scope("train"):
    # Get gradients of all trainable variables
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)


In [11]:
# Add gradients to summary
for gradient, var in gradients:
    tf.summary.histogram(var.name.replace(":", "_") + '/gradient', gradient)

# Add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name.replace(":", "_"), var)

# Add the loss to summary
tf.summary.scalar('Loss', loss)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Add the accuracy to the summary
tf.summary.scalar('Accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()


In [12]:
# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

In [13]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = int(np.floor(tr_data.data_size/batch_size))
val_batches_per_epoch = int(np.floor(val_data.data_size / batch_size))

print(train_batches_per_epoch, val_batches_per_epoch)


312 31


In [14]:
# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path + '/finetune1/')

fetch_time=0
train_time=0

# Start Tensorflow session
with tf.Session() as sess:

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)

    # Load the pretrained weights into the non-trainable layer
    model.load_initial_weights(sess)

    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(),
                                                      filewriter_path))

    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            
            st=time.time()
            img_batch, label_batch = sess.run(next_batch)
            print(img_batch.shape)
            et=time.time()
            
            fetch_time+=(et - st)
            
            print('fetching batch time: %.3f' % (et - st))
            
            st1=time.time()
            # And run the training op
            sess.run(train_op, feed_dict={x: img_batch,
                                          y: label_batch,
                                          keep_prob: dropout_rate})
            et1=time.time()
            
            train_time+=(et1 - st1)
            print('training time: %.3f' % (et1 - st1))
            
            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch,
                                                        keep_prob: 1.})

                writer.add_summary(s, epoch*train_batches_per_epoch + step)

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op)
        test_acc = 0.
        test_count = 0
        for _ in range(val_batches_per_epoch):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch,
                                                keep_prob: 1.})
            test_acc += acc
            test_count += 1
            
        test_acc /= test_count
        
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        print("{} Saving checkpoint of model...".format(datetime.now()))

        # save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path,
                                       'finetune-model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                       checkpoint_name))

    print('total time spent on fetching data:', fetch_time)
    print('total time spent on training data:', train_time)

2018-05-06 23:21:44.052982 Start training...
2018-05-06 23:21:44.053065 Open Tensorboard at --logdir /tmp/finetune_alexnet/tensorboard
2018-05-06 23:21:44.053088 Epoch number: 1
(32, 227, 227, 3)
fetching batch time: 1.709
training time: 1.930
(32, 227, 227, 3)
fetching batch time: 0.022
training time: 0.071
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.112
(32, 227, 227, 3)
fetching batch time: 0.042
training time: 0.089
(32, 227, 227, 3)
fetching batch time: 0.005
training time: 0.087
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.081
(32, 227, 227, 3)
fetching batch time: 0.019
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.087
(32, 227, 227, 3)
fetching bat

training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.079
(32, 227, 227, 3)
fetching batch time: 0.017
training time: 0.131
(32, 227, 227, 3)
fetching batch time: 0.031
training time: 0.108
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.087
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.089
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.019
training time: 0.081
(32, 227, 227, 3)
fetching batch time: 0.012
training t

training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training t

training time: 0.081
(32, 227, 227, 3)
fetching batch time: 0.006
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.034
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.023
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.013
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.016
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.071
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.025
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.013
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.071
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.076
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.010
training t

training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.005
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.003
training t

...

(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.079
(32, 227, 227, 3)
fetching batch time: 0.016
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.014
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.023
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.071
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.024
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.076
(32, 227, 

training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.007
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training t

training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.017
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.018
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.076
(32, 227, 227, 3)
fetching batch time: 0.037
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.078
(32, 227, 227, 3)
fetching batch time: 0.015
training time: 0.082
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.087
(32, 227, 227, 3)
fetching batch time: 0.006
training time: 0.076
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.015
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.012
training t

(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.009
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.071
(32, 227, 227, 3)
fetching batch time: 0.012
training time: 0.077
(32, 227, 227, 3)
fetching batch time: 0.022
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.012
training time: 0.079
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.073
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.078
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.075
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.074
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.020
training time: 0.069
(32, 227, 

training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.005
training time: 0.072
(32, 227, 227, 3)
fetching batch time: 0.011
training time: 0.070
(32, 227, 227, 3)
fetching batch time: 0.004
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.010
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.003
training time: 0.068
(32, 227, 227, 3)
fetching batch time: 0.008
training time: 0.069
(32, 227, 227, 3)
fetching batch time: 0.004
training t

In [15]:
# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path + '/fromscratch1/')

fetch_time=0
train_time=0

# Start Tensorflow session
with tf.Session() as sess:

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)

    # Load the pretrained weights into the non-trainable layer
    # model.load_initial_weights(sess)

    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(),
                                                      filewriter_path))

    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            
            st=time.time()
            img_batch, label_batch = sess.run(next_batch)
            et=time.time()
            
            fetch_time+=(et - st)
            
            print('fetching batch time: %.3f' % (et - st))
            
            st1=time.time()
            # And run the training op
            sess.run(train_op, feed_dict={x: img_batch,
                                          y: label_batch,
                                          keep_prob: dropout_rate})
            et1=time.time()
            
            train_time+=(et1 - st1)
            print('training time: %.3f' % (et1 - st1))
            
            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch,
                                                        keep_prob: 1.})

                writer.add_summary(s, epoch*train_batches_per_epoch + step)

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op)
        test_acc = 0.
        test_count = 0
        for _ in range(val_batches_per_epoch):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch,
                                                keep_prob: 1.})
            test_acc += acc
            test_count += 1
            
        test_acc /= test_count
        
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        print("{} Saving checkpoint of model...".format(datetime.now()))

        # save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path,
                                       'scratch-model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                       checkpoint_name))

    print('total time spent on fetching data:', fetch_time)
    print('total time spent on training data:', train_time)

2018-05-06 23:31:13.146206 Start training...
2018-05-06 23:31:13.220032 Open Tensorboard at --logdir /tmp/finetune_alexnet/tensorboard
2018-05-06 23:31:13.220095 Epoch number: 1
fetching batch time: 1.938
training time: 0.116
fetching batch time: 0.011
training time: 0.077
fetching batch time: 0.003
training time: 0.072
fetching batch time: 0.007
training time: 0.070
fetching batch time: 0.016
training time: 0.080
fetching batch time: 0.005
training time: 0.069
fetching batch time: 0.008
training time: 0.071
fetching batch time: 0.009
training time: 0.078
fetching batch time: 0.012
training time: 0.068
fetching batch time: 0.018
training time: 0.072
fetching batch time: 0.008
training time: 0.074
fetching batch time: 0.009
training time: 0.082
fetching batch time: 0.012
training time: 0.073
fetching batch time: 0.004
training time: 0.071
fetching batch time: 0.030
training time: 0.076
fetching batch time: 0.010
training time: 0.069
fetching batch time: 0.024
training time: 0.069
fetchi

training time: 0.072
fetching batch time: 0.011
training time: 0.075
fetching batch time: 0.009
training time: 0.070
fetching batch time: 0.008
training time: 0.069
fetching batch time: 0.007
training time: 0.069
fetching batch time: 0.008
training time: 0.072
fetching batch time: 0.009
training time: 0.073
fetching batch time: 0.004
training time: 0.075
fetching batch time: 0.008
training time: 0.069
fetching batch time: 0.021
training time: 0.073
fetching batch time: 0.008
training time: 0.069
fetching batch time: 0.007
training time: 0.078
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time

training time: 0.079
fetching batch time: 0.009
training time: 0.080
fetching batch time: 0.030
training time: 0.078
fetching batch time: 0.013
training time: 0.072
fetching batch time: 0.031
training time: 0.070
fetching batch time: 0.040
training time: 0.072
fetching batch time: 0.015
training time: 0.069
fetching batch time: 0.014
training time: 0.069
fetching batch time: 0.013
training time: 0.076
fetching batch time: 0.009
training time: 0.077
fetching batch time: 0.026
training time: 0.070
fetching batch time: 0.020
training time: 0.072
fetching batch time: 0.008
training time: 0.072
fetching batch time: 0.019
training time: 0.069
fetching batch time: 0.035
training time: 0.069
fetching batch time: 0.006
training time: 0.073
fetching batch time: 0.018
training time: 0.072
fetching batch time: 0.026
training time: 0.069
fetching batch time: 0.005
training time: 0.071
fetching batch time: 0.027
training time: 0.069
fetching batch time: 0.011
training time: 0.069
fetching batch time

training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.005
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.009
training time: 0.068
fetching batch time: 0.005
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.005
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time

training time: 0.070
fetching batch time: 0.013
training time: 0.068
fetching batch time: 0.021
training time: 0.068
fetching batch time: 0.010
training time: 0.069
fetching batch time: 0.016
training time: 0.070
fetching batch time: 0.021
training time: 0.074
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.010
training time: 0.073
fetching batch time: 0.011
training time: 0.074
fetching batch time: 0.005
training time: 0.074
fetching batch time: 0.014
training time: 0.074
fetching batch time: 0.016
training time: 0.069
fetching batch time: 0.008
training time: 0.075
fetching batch time: 0.013
training time: 0.069
fetching batch time: 0.012
training time: 0.073
fetching batch time: 0.008
training time: 0.069
fetching batch time: 0.020
training time: 0.068
fetching batch time: 0.012
training time: 0.072
fetching batch time: 0.010
training time: 0.080
fetching batch time: 0.011
training time: 0.069
fetching batch time: 0.011
training time: 0.071
fetching batch time

...

training time: 0.075
fetching batch time: 0.008
training time: 0.078
fetching batch time: 0.020
training time: 0.075
fetching batch time: 0.180
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.003
training time: 0.068
fetching batch time: 0.005
training time: 0.069
fetching batch time: 0.011
training time: 0.069
fetching batch time: 0.015
training time: 0.075
fetching batch time: 0.005
training time: 0.069
fetching batch time: 0.006
training time: 0.069
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time

training time: 0.076
fetching batch time: 0.017
training time: 0.068
fetching batch time: 0.009
training time: 0.076
fetching batch time: 0.012
training time: 0.069
fetching batch time: 0.047
training time: 0.076
fetching batch time: 0.014
training time: 0.076
fetching batch time: 0.013
training time: 0.068
fetching batch time: 0.019
training time: 0.070
fetching batch time: 0.007
training time: 0.084
fetching batch time: 0.048
training time: 0.171
fetching batch time: 0.009
training time: 0.069
fetching batch time: 0.025
training time: 0.078
fetching batch time: 0.023
training time: 0.099
fetching batch time: 0.026
training time: 0.075
fetching batch time: 0.016
training time: 0.073
fetching batch time: 0.012
training time: 0.071
fetching batch time: 0.020
training time: 0.077
fetching batch time: 0.007
training time: 0.083
fetching batch time: 0.021
training time: 0.078
fetching batch time: 0.026
training time: 0.069
fetching batch time: 0.011
training time: 0.080
fetching batch time

training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time

training time: 0.080
fetching batch time: 0.012
training time: 0.068
fetching batch time: 0.031
training time: 0.071
fetching batch time: 0.013
training time: 0.068
fetching batch time: 0.025
training time: 0.072
fetching batch time: 0.014
training time: 0.083
fetching batch time: 0.011
training time: 0.069
fetching batch time: 0.024
training time: 0.076
fetching batch time: 0.012
training time: 0.072
fetching batch time: 0.011
training time: 0.072
fetching batch time: 0.009
training time: 0.077
fetching batch time: 0.015
training time: 0.075
fetching batch time: 0.012
training time: 0.069
fetching batch time: 0.013
training time: 0.073
fetching batch time: 0.022
training time: 0.070
fetching batch time: 0.011
training time: 0.069
fetching batch time: 0.042
training time: 0.071
fetching batch time: 0.011
training time: 0.072
fetching batch time: 0.011
training time: 0.078
fetching batch time: 0.014
training time: 0.084
fetching batch time: 0.011
training time: 0.075
fetching batch time

training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.006
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.008
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time: 0.004
training time: 0.068
fetching batch time: 0.007
training time: 0.068
fetching batch time

In [ ]:
train_file

with open(train_file, "r") as fp:
    datasets_=fp.readlines()
print(len(datasets_))

with open(val_file, "r") as fp:
    datasets_=fp.readlines()
print(len(datasets_))

# for i in datasets_:
#     print(i.split('\n')[0])